In [2]:
import requests
import os
import pandas as pd
tokenCanvas = os.environ['tokenCanvas']

FSbrukar = os.environ["FSbrukar"]
FSpassord = os.environ["FSpassord"]

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}


In [11]:
def query_FS_graphql(query, variable):
    hode = {
        'Accept': 'application/json;version=1',
        "Feature-Flags": "beta, experimental"
    }
    GraphQLurl = "https://api.fellesstudentsystem.no/graphql/"
    svar = requests.post(
        GraphQLurl, 
        json = {
            'query': query,
            'variables': variable
        },
        headers=hode,
        auth=(FSbrukar, FSpassord))
    if 200 <= svar.status_code < 300:
        return svar.json()
    else:
        return 

query = """
query MyQuery($personlopenumre: [String!]!, $eierOrganisasjonskode: String!) {
  personProfilerGittPersonlopenumre(
    personlopenumre: $personlopenumre
    eierOrganisasjonskode: $eierOrganisasjonskode
  ) {
    navn {
      etternavn
      fornavn
    }
    institusjonsEpost
  }
}
"""

## Les inn data
"JML" er lista med namn som vi ikkje finn i Canvas.

In [12]:
jml = pd.read_csv("JML.csv")
jml.head()

,id,epost
0,41244,jorunn.kristin.aa.albretsen@hvl.no
1,34042,marit.eva.solheim@hvl.no
2,863674,chmol2389@hvl.no
3,936058,markus.steen@hvl.no
4,247006,torespen.kristensen@hvl.no


In [33]:
for i, r in jml.iterrows():
    print(f"{r['id']}:{r['epost']}:", end="")
    variable = {
        "personlopenumre": [str(r['id'])],
        "eierOrganisasjonskode": "203"
    }
    try:
        svar = query_FS_graphql(query, variable)
        FS_epost = svar['data']['personProfilerGittPersonlopenumre'][0]['institusjonsEpost']
    except:
        FS_epost = None
    if FS_epost is None:
        print(f"not_found:")
        continue
    else:
        print(f"{FS_epost}:", end="")
    url1 = f"https://hvl.instructure.com/api/v1/accounts/1/users?search_term=fs:203:{r['id']}"
    response1 = requests.get(url1, headers=headers, params=params)
    if response1.json() == []:
        print(f"not_found")
        continue
    else:
        url2 = f"https://hvl.instructure.com/api/v1/users/{response1.json()[0]['id']}"
        response2 = requests.get(url2, headers=headers, params=params)
        print(f"{response2.json()['email']}")

41244:jorunn.kristin.aa.albretsen@hvl.no:092071@stud.hvl.no:092071@stud.hvl.no
34042:marit.eva.solheim@hvl.no:marit.eva.solheim@hvl.no:not_found
863674:chmol2389@hvl.no:christmo@hvl.no:christmo@hvl.no
936058:markus.steen@hvl.no:markus.steen@sintef.no:not_found
247006:torespen.kristensen@hvl.no:not_found:
29659:gunstein.ersland.pytte@hvl.no:gunstein.ersland.pytte@hvl.no:not_found
235295:signe.berit.bentsen@hvl.no:signe.bentsen@hvl.no:signe.bentsen@hvl.no
172775:torbjorn.heyerdahl@hvl.no:torbjorn.sigurd.heyerdahl@hvl.no:torbjorn.sigurd.heyerdahl@hvl.no
56972:kristin.vassbotn.guldhav@hvl.no:040400@stud.hvl.no:040400@stud.hvl.no
31818:inge.heldal.thorsen@hvl.no:inge.thorsen@hvl.no:inge.thorsen@hvl.no
33403:jonas.anton.christophersen@hvl.no:jonas.a.christophersen@hvl.no:jonas.a.christophersen@hvl.no
103825:kristina.skogly@hvl.no:041052@stud.hvl.no:667083@stud.hvl.no
40583:siri.wolneberg.bothun@hvl.no:siri@sbnatur.no:144691@stud.hvl.no
515737:tesfa.yigrem.mengestie@hvl.no:tesfa.mengestie@hvl

In [31]:
svar

{'data': {'personProfilerGittPersonlopenumre': [None]}}